In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import load_img
import numpy as np

In [ ]:


def remove_top_layer(model):
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []

class AlikeFinder():
    def __init__(self):
        self.prepare_model()
        
    def prepare_model(self):
        model = InceptionV3(weights='imagenet')
        remove_top_layer(model)
        
        self.model = model
        
original_model = InceptionV3(weights='imagenet')
alike_finder = AlikeFinder()        

In [ ]:
test_image_path = '/home/prd/Development/projects/tensorflow/1.Dogs-Cats/data/playground/train/dog/dog.107.jpg' 
image = load_img(test_image_path)
image = np.array(image).astype(np.uint8)
image = np.reshape(image, (1, image.shape[0], image.shape[1], image.shape[2]))
result = alike_finder.model.predict(image)
orig_result = original_model.predict(image)

In [ ]:
print(result.shape)
